In [2]:
import re
import os
import numpy as np
import pandas as pd
import csv
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import load_model, Sequential, Model
from keras.layers import Dense, Dropout, Softmax, BatchNormalization, Embedding, LSTM, SimpleRNN
from keras.initializers import GlorotNormal, GlorotUniform
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder

In [92]:
import spacy
# !python -m spacy download en_core_web_lg
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_lg")

^C


In [93]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [94]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


In [95]:
PATH_TO_SAVE = "./data/lyrics_processed2.csv"

## Song Classifier

### RNN (LSTM)

Dataset: https://www.kaggle.com/neisse/scrapped-lyrics-from-6-genres

#### Import Data


In [96]:
base_dir = './data'
artists_df = pd.read_csv(f'{base_dir}/artists-data.csv')
lyrics_df = pd.read_csv(f'{base_dir}/lyrics-data.csv')

In [97]:
artists_df.head()

,Artist,Songs,Popularity,Link,Genre,Genres
0,10000 Maniacs,110,0.3,/10000-maniacs/,Rock,Rock; Pop; Electronica; Dance; J-Pop/J-Rock; G...
1,12 Stones,75,0.3,/12-stones/,Rock,Rock; Gospel/Religioso; Hard Rock; Grunge; Roc...
2,311,196,0.5,/311/,Rock,Rock; Surf Music; Reggae; Ska; Pop/Rock; Rock ...
3,4 Non Blondes,15,7.5,/4-non-blondes/,Rock,Rock; Pop/Rock; Rock Alternativo; Grunge; Blue...
4,A Cruz Está Vazia,13,0.0,/a-cruz-esta-vazia/,Rock,Rock


In [98]:
lyrics_df.head()

,ALink,SName,SLink,Lyric,Idiom
0,/10000-maniacs/,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH
1,/10000-maniacs/,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH
2,/10000-maniacs/,These Are Days,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH
3,/10000-maniacs/,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH
4,/10000-maniacs/,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,ENGLISH


#### Preprocessing Data

In [99]:
lyrics_eng_df = lyrics_df[lyrics_df["Idiom"] == "ENGLISH"]
artists_df = artists_df[["Artist", "Link", "Genre"]]

In [100]:
### Pull unique artists
print(artists_df.shape)
artists_df.duplicated().value_counts()

(3242, 3)


False    3242
dtype: int64

In [101]:
artists_df["Genre"].value_counts()

Rock            797
Pop             796
Sertanejo       617
Hip Hop         537
Funk Carioca    302
Samba           193
Name: Genre, dtype: int64

In [102]:
artists_df[artists_df["Genre"] == "Funk Carioca"] #this gnre would be probably supress in the join

,Artist,Link,Genre
412,3zi,/3zi/,Funk Carioca
580,Mc God,/mc-god/,Funk Carioca
581,MC Guga da VG,/mc-guga-da-vg/,Funk Carioca
582,Mc Leoest,/mc-leoest/,Funk Carioca
583,Mc Patrick MF,/mc-patrick-mf/,Funk Carioca
...,...,...,...
3173,Jeann & Julio,/jeann-julio/,Funk Carioca
3183,Luiz Henrique & Fernando,/luiz-henrique-e-fernando/,Funk Carioca
3184,Marcos e Fernando,/marcos-e-fernando/,Funk Carioca
3188,Pedro Henrique e Fernando,/pedro-henrique-e-fernando/,Funk Carioca


In [103]:
lyrics_eng_df.head()


,ALink,SName,SLink,Lyric,Idiom
0,/10000-maniacs/,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH
1,/10000-maniacs/,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH
2,/10000-maniacs/,These Are Days,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH
3,/10000-maniacs/,A Campfire Song,/10000-maniacs/a-campfire-song.html,"A lie to say, ""O my mountain has coal veins an...",ENGLISH
4,/10000-maniacs/,Everyday Is Like Sunday,/10000-maniacs/everyday-is-like-sunday.html,Trudging slowly over wet sand. Back to the ben...,ENGLISH


In [104]:
lyrics_eng_df.duplicated().value_counts()


False    91622
True     23101
dtype: int64

In [105]:
lyrics_eng_df.drop_duplicates(subset='SLink', inplace=True, keep='first', ignore_index=False)
lyrics_eng_df.duplicated().value_counts()

C:\Users\mazg9\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


False    91611
dtype: int64

In [106]:
lyrics_df_cln = lyrics_eng_df.drop(['SName', 'SLink', 'Idiom'], axis=1)
lyrics_df_cln

,ALink,Lyric
0,/10000-maniacs/,I could feel at the time. There was no way of ...
1,/10000-maniacs/,"Take me now, baby, here as I am. Hold me close..."
2,/10000-maniacs/,These are. These are days you'll remember. Nev...
3,/10000-maniacs/,"A lie to say, ""O my mountain has coal veins an..."
4,/10000-maniacs/,Trudging slowly over wet sand. Back to the ben...
...,...,...
207611,/sambo/,"(Chorus). Hello, hello,hello,how low. Hello,he..."
207624,/sambo/,Well sometimes I go out by myself. And I look ...
207628,/sambo/,Feeling my way through the darkness. Guided by...
207792,/seu-jorge/,"Don't, don't, that's what you say. Each time t..."


In [107]:
lyrics_df_cln.duplicated().value_counts() #there is 219 duplicated songs

False    91392
True       219
dtype: int64

In [108]:
lyrics_df_cln.drop_duplicates(inplace=True)

lyrics_df_cln.duplicated().value_counts() #there is 219 duplicated songs

False    91392
dtype: int64

In [109]:
### Join Data Frames

X = pd.merge(lyrics_df_cln, artists_df, how='inner', left_on='ALink', right_on='Link')
X = X.drop(["ALink", "Link"], axis=1)
X.Genre.value_counts()

Rock            50048
Pop             28355
Hip Hop         16115
Funk Carioca       69
Sertanejo          58
Samba              42
Name: Genre, dtype: int64

In [110]:
### Drop imbalanced genres

X = X[~X["Genre"].isin(["Funk Carioca", "Sertanejo", "Samba"])]
X.Genre.value_counts()

Rock       50048
Pop        28355
Hip Hop    16115
Name: Genre, dtype: int64

In [111]:
def get_num_unique_words(s):
    corpus = s.str.cat(sep=" ")
    corpus = re.sub(r'[^a-zA-Z0-9\s{1}áéíóúüñÁÉÍÓÚ]', '', corpus).lower().strip().rstrip('\r\n').rstrip('\n')
    unique_words = set(corpus.split(" "))
    return len(unique_words)
get_num_unique_words(X["Lyric"])

154874

In [112]:
X["Lyric"].str.len().sort_values()

42021        9
76345        9
33200       15
24529       16
84174       17
         ...  
26685    19578
5100     32971
26400    33998
22909    41494
2997     64047
Name: Lyric, Length: 94518, dtype: int64

In [113]:
X[X["Lyric"].str.len() < 100].iloc[0]["Lyric"]

"(I trusted you..). Wake up. I've been... watching you.."

In [ ]:
# class LemmaTokenizer: 
#     def __init__(self, nlp):
#         self.nlp = nlp
#     def __call__(self, doc):
#         return list(filter(None, [t.lemma_ if not t.is_stop else None for t in self.nlp(doc)]))

# tokenizer = LemmaTokenizer(nlp)
# X["LyricNorm"] = X["Lyric"].apply(tokenizer)

2021-09-24 15:42:05.507427: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
X["LyricNormStr"] = X["LyricNorm"].apply(lambda x: ' '.join(x))
X["LyricNormStr"]

In [ ]:
df=X[["Lyric", "LyricNormStr", "Artist", "Genre"]]
df

In [ ]:
df.to_csv(PATH_TO_SAVE)

In [7]:
X = pd.read_csv('./data/lyrics_processed2.csv')

In [ ]:
size = get_num_unique_words(X["LyricNormStr"])
size

In [8]:
VOCAB_SIZE = 10000
EMBEDDING_DIM = 32
MAX_LENGTH = 128
TRUNC_TYPE = 'post'
PADDING_TYPE = 'post'
OOV_TOK = "<OOV>"
EPOCHS = 5

In [9]:
VOCAB_SIZE

10000

In [10]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token = OOV_TOK)
tokenizer.fit_on_texts(X["LyricNormStr"])

word_index = tokenizer.word_index

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X.loc[:, X.columns != 'Genre'], X['Genre'], test_size = 0.3, random_state = 5)

In [12]:
y_train

28246       Rock
70272    Hip Hop
73358        Pop
67799    Hip Hop
26305       Rock
          ...   
59465    Hip Hop
5520        Rock
20463       Rock
18638       Rock
35683       Rock
Name: Genre, Length: 66162, dtype: object

In [13]:
training_seq = tokenizer.texts_to_sequences(X_train["Lyric"])
training_pd = pad_sequences(training_seq, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNC_TYPE)

In [14]:
testing_seq = tokenizer.texts_to_sequences(X_test["Lyric"])
testing_pd = pad_sequences(testing_seq, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNC_TYPE)

#### One Hot Encoding Labels

In [15]:
ohe = OneHotEncoder(sparse=False)
training_lbls = ohe.fit_transform(y_train.to_numpy().reshape(-1,1))
training_lbls


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [16]:

testing_lbls = ohe.fit_transform(y_test.to_numpy().reshape(-1,1))
testing_lbls


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [20]:
X_train["Artist"]

28246               Natalie Merchant
70272                   Willow Smith
73358                  Big Time Rush
67799                         T-Pain
26305                          McFly
                    ...             
59465                       Ice Cube
5520                         Blondie
20463    Joan Jett & The Blackhearts
18638                Imagine Dragons
35683                    Rod Stewart
Name: Artist, Length: 66162, dtype: object

In [17]:
ohe.get_feature_names()

array(['x0_Hip Hop', 'x0_Pop', 'x0_Rock'], dtype=object)

In [18]:
training_pd = np.array(training_pd)
testing_pd = np.array(testing_pd)

In [28]:
training_x = X_train["Artist"].to_numpy()
testing_x = X_test["Artist"].to_numpy()

#### Save Numpy Objects

In [31]:
with open('./data/train.npy', 'wb') as f:
    np.save(f, training_pd)
    np.save(f, training_lbls)
    np.save(f, VOCAB_SIZE)
    
with open('./data/test.npy', 'wb') as f:
    np.save(f, testing_pd)
    np.save(f, testing_lbls)